# Learning from Data Week 1

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from numpy.random import uniform

# Set randomizer seed
np.random.seed(0)

## Perceptron Algorithm

In [3]:
# Generate random points
def generate_points(N):
    """
    Input: number of random points to be generated in [-1,1]x[-1,1]
    Output: coordinates of points, adding x0 = 1's
    """
    xs = uniform(-1,1,N)
    ys = uniform(-1,1,N)
    # Generate vector of ones in the 0th column
    return np.array(zip(np.ones(N), xs, ys))

points = generate_points(100)

In [4]:
# Create function f and assign positives and negatives
def assignment(points):
    """
    Input: random points
    Output: positive and negative labels, according to random linear function f that separates these points
    """
    # Randomly select 2 2-D points and 1 direction
    (x1, y1, x2, y2, direction) = uniform(-1,1,5)
    # Calculate slope and intercept
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m*x1
    # Convert to a weight vector
    w = [-b, -m * np.sign(direction), 1*np.sign(direction)]
    #print 'Random function f:'
    #print 'w = {}'.format(w)    
    # Label points
    labels = np.sign(np.dot(points, w))    

    # Plot of positive and negative examples of random linear function f
    #plt.figure(figsize = (12,12))
    #plt.plot(points[labels == 1][:,1], points[labels == 1][:,2], 'bo')
    #plt.plot(points[labels == -1][:,1], points[labels == -1][:,2], 'ro')
    return (w, labels)
    
w, labels = assignment(points)

In [5]:
# Function to calculate probability of P(f != g):
def probs(w, w_est):
    """
    Input: actual weights and estimated weights
    Output: probability in [-1,1]x[-1,1] that the classifications disagree
    """
    # Create grid of points
    x, y = np.linspace(-1, 1, 100), np.linspace(-1, 1, 100)
    xv, yv = np.meshgrid(x, y)
    grid = np.array(zip(xv.flatten(), yv.flatten()))
    
    # Calculate actual and predicted labels
    actual_labels = np.sign(np.dot(points, w))
    predicted_labels = np.sign(np.dot(points, w_est))

    # Calculate probability of nonoverlap
    return np.mean(actual_labels != predicted_labels)

In [6]:
# Run PLA till convergence
def PLA(N):
    """
    Input: number of random points generated
    Output: number of iterations till convergence, and probaility of error
    """
    
    # Generate training examples and labels
    points = generate_points(N)
    w, labels = assignment(points)
    # Initialize weights and iteration count
    w_est = np.array([0, 0, 0])
    iters = 0
    accuracy = 0
    
    # Iterate to improve weights
    while accuracy < 1:
        # Make and evaluate predictions
        preds = np.sign(np.dot(points, w_est))
        wrong_points = points[preds != labels]
        wrong_labels = labels[preds != labels]
        
        # Measure accuracy of current iteration
        accuracy = np.mean(preds == labels)
        
        # Randomly pick one misclassified point and update weight
        if accuracy < 1:
            index = np.random.randint(wrong_points.shape[0])
            w_est = w_est + wrong_labels[index] * wrong_points[index]        
        iters += 1
        
    # Compare to function f
    #print ('Estimated function g:')
    #print ('w_actual = {}'.format(w))
    #print ('w_est = {}'.format(w_est))
    #print ('Number of iterations = {:.3f}'.format(iters))
    # Calculate error
    error = probs(w, w_est)
    #print ('P(f!=g) = {:.3f}'.format(error))
    return (iters, error)

PLA(10)

(6, 0.13)

In [9]:
# Repeat experiment 1000 times
def experiment(times, N):
    iters_total = []
    error_total = []
    for run in range(times):
        iters, error = PLA(N)
        iters_total.append(iters)
        error_total.append(error)
    return np.mean(iters_total), np.mean(error_total)

# N = 10
experiment(1000, 10)

(11.805999999999999, 0.10832000000000003)

In [10]:
# N = 100
experiment(1000, 100)

(103.35899999999999, 0.01354)